In [1]:
import cvxpy as cp
import numpy as np

# Placement with QCQP
**Inputs:** $h_1, \dots, h_{n}, \mu_1, \dots, \mu_n$ (disk heats and throughputs) \
**Output:** Matrix $P = (p_{ij})$, $p_{ij} = 1$ IFF block $i$ is placed to disk $j$ \
**Problem:** Minimize
$ X = \sum_{i=1}^{N} \left(  \sum_{j=1}^{n} p_{i_j}\mu_i h_j - \sum_{k=1}^{N}\sum_{j=1}^{n}  \frac{\mu_k  h_j}{N}   p_{k_j} \right)^2 $ where
$ \tilde{H} =  \frac{1}{N} \sum_{k=1}^{N} \mu_k \left( \sum_{j=1}^{n} p_{k_j} h_j\right)$ \
**Constraints:**
$p_{i_1} + \dots + p_{i_{N}} = 1 $


In [56]:
def get_x(mu, h):
    def pos(i, j):
        return j * N + i
    
    N = len(mu)
    n = len(h)
    
    A = []
    for i in range(N):
        curr = np.zeros(N * n)
        for j in range(n):
            curr[pos(i, j)] = mu[i] * h[j]
        A.append(curr)
    A = np.array(A) 

    B = []
    curr = np.zeros(N * n)
    for k in range(N):
        for j in range(n):
            curr[pos(k, j)] = (mu[k] * h[j]) / N
    for i in range(N):
        B.append(curr)
    B = np.array(B)

    C = []
    for j in range(n):
        curr = np.zeros(N * n)
        for i in range(N):
            curr[pos(i, j)] = 1
        C.append(curr)
    C = np.array(C)
    
    x = cp.Variable(n * N, integer=True)
    objective = cp.Minimize(cp.sum_squares(A @ x - B @ x))
    constraints = [0 <= x, x <= 1, C @ x == np.ones(n)]
    prob = cp.Problem(objective, constraints)

    
    result = prob.solve()

    return x

In [57]:
N = 4
m = 100
n = N * m
mu = np.ones(N) # disk bandwidths
h = np.zeros(n) # block heats

for k in range(m):
    h[N * k] = 1

x = get_x(mu, h)

In [58]:
def output_placement(x, n, N):
    for j in range(n):
        for i in range(N):
            if x.value[pos(i, j)] == 1:
                print('Block {} is placed to disk {}'.format(j, i))

In [79]:
def get_placement(m=100, N=4):
    def pos(i, j):
        return j * N + i
    
    n = N * m
    placement = np.ones(n, dtype=int)
    
    mu = np.ones(N) # case of heteroheous disks
    
    for res in range(N): # split into 4 clusters and find placements separately
        h = np.zeros(n)
        for k in range(m):
            h[N * k + res] = 1
            
        x = get_x(mu, h)
        for k in range(m):
            j = N * k + res # find position of this block
            for i in range(N):
                if x.value[pos(i, j)] == 1:
                    placement[j] = i
    return placement
        

In [94]:
N = 4
m = 1024
placement = get_placement(m, N)

for res in range(N):
    arr = np.zeros(N, dtype=int)
    for k in range(m):
        j = N * k + res
        arr[placement[j]] += 1
    print(arr)

[256 256 256 256]
[256 256 256 256]
[256 256 256 256]
[256 256 256 256]
[3 3 3 ... 3 3 3]


In [99]:
f = open('synthetic_placement.txt', 'a')
for a in placement:
    f.write('{} '.format(a))
f.close()

In [63]:
for i in range(N):
    disk_content = []
    for j in range(n):
        if x.value[pos(i, j)] == 1:
            disk_content.append(j)
    print('{}: {}'.format(i, disk_content))

0: [1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14, 15, 17, 18, 19, 21, 22, 23, 25, 26, 27, 29, 30, 31, 33, 34, 35, 37, 38, 39, 41, 42, 43, 45, 46, 47, 49, 50, 51, 53, 54, 55, 57, 58, 59, 61, 62, 63, 65, 66, 67, 69, 70, 71, 73, 74, 75, 77, 78, 79, 81, 82, 83, 85, 86, 87, 89, 90, 91, 93, 94, 95, 97, 98, 99, 101, 102, 103, 105, 106, 107, 109, 110, 111, 113, 114, 115, 117, 118, 119, 121, 122, 123, 125, 126, 127, 129, 130, 131, 133, 134, 135, 137, 138, 139, 141, 142, 143, 145, 146, 147, 149, 150, 151, 153, 154, 155, 157, 158, 159, 161, 162, 163, 165, 166, 167, 169, 170, 171, 173, 174, 175, 177, 178, 179, 181, 182, 183, 185, 186, 187, 189, 190, 191, 193, 194, 195, 197, 198, 199, 201, 202, 203, 205, 206, 207, 209, 210, 211, 213, 214, 215, 217, 218, 219, 221, 222, 223, 225, 226, 227, 229, 230, 231, 233, 234, 235, 237, 238, 239, 241, 242, 243, 245, 246, 247, 249, 250, 251, 253, 254, 255, 257, 258, 259, 261, 262, 263, 265, 266, 267, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283